**Avant de lancer ce notebook merci de générer les csv à l'aide du [notebook suivant](Create-csv-files.ipynb).**

In [1]:
# Commenter la configuration non utilisée

# Configuration production
#mongoDBUrl = 'mongodb017.westeurope.cloudapp.azure.com'
#port = 32002

# Configuration dev
mongoDBUrl = 'localhost'
port = 27017

In [2]:
import pandas as pd
import numpy as np
import json
from pymongo import MongoClient

In [3]:
client = MongoClient( mongoDBUrl, port )
db = client['db_credit']

In [4]:
def drop_collections(client, db):
    
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        collection.drop()

In [5]:
drop_collections(client, db)

Dénormalisation retenue:

In [6]:
{
    "members":
    {
        "member_no": "1",
        "lastname": "DOUGLAS",
        "firstname": "Michael",
        "phone_number":"blabla",

        "location":
        {
            "country":"France",
            "region_no":"IDF",
            "city":"Paris",
            "mail_code":"75000",
            "street":"blabla"
        },

        "balance":
        {
            "prev_balance": "1000",
            "curr_balance": "1200"
        },

        "_comment":"No need to have a list, each employee have only 1 corporation",
        "corporation":
        {
            "corp_no":1,
            "corp_name":"blabla",
            "region_no":"XX"
        },

        "charges": [
            {
                "charge_no": "125",
                "category_no": "12",
                "category_desc": "Travel",
                "charge_dt": "2000-02-22 10:43:38",
                "charge_amount": "336.0000",

                "provider_no":12,
                "provider_name":"BNP Paribas",
                "provider_phone_number": "blabla",
                "payments":[1546, 1547,... , 1578]
            }
        ]
    },
     "providers":
        {
            "provider_no":1,
            "provider_name":"BNP",
            "location":
            {
                "country":"France",
                "region_no":"IDF",
                "city":"Paris",
                "mail_code":"75000"
            }
        },
 
 
    "payments":
        {
            "payment_no":1546,
            "payment_dt":"2000-02-22 10:43:38",
            "payment_amt":125.00
        }
}


{'members': {'member_no': '1',
  'lastname': 'DOUGLAS',
  'firstname': 'Michael',
  'phone_number': 'blabla',
  'location': {'country': 'France',
   'region_no': 'IDF',
   'city': 'Paris',
   'mail_code': '75000',
   'street': 'blabla'},
  'balance': {'prev_balance': '1000', 'curr_balance': '1200'},
  '_comment': 'No need to have a list, each employee have only 1 corporation',
  'corporation': {'corp_no': 1, 'corp_name': 'blabla', 'region_no': 'XX'},
  'charges': [{'charge_no': '125',
    'category_no': '12',
    'category_desc': 'Travel',
    'charge_dt': '2000-02-22 10:43:38',
    'charge_amount': '336.0000',
    'provider_no': 12,
    'provider_name': 'BNP Paribas',
    'provider_phone_number': 'blabla',
    'payments': [1546, 1547, Ellipsis, 1578]}]},
 'providers': {'provider_no': 1,
  'provider_name': 'BNP',
  'location': {'country': 'France',
   'region_no': 'IDF',
   'city': 'Paris',
   'mail_code': '75000'}},
 'payments': {'payment_no': 1546,
  'payment_dt': '2000-02-22 10:43:3

In [7]:
Members = pd.read_csv('Members.csv')
Corporations = pd.read_csv('Corporations.csv')
Categories = pd.read_csv('Categories.csv')
Providers = pd.read_csv('Prodivers.csv')
Charges = pd.read_csv('Charges.csv')
Statements = pd.read_csv('Statements.csv')
Payments = pd.read_csv('Payments.csv')

In [8]:
Charges

,Charge type,Charge amount,Duration,dt,member_no,provider_no,charge_no
0,0,100696,36,2021-10-18,4489,21,0
1,0,71684,60,2021-07-18,7342,36,1
2,1,500000,48,2021-09-05,159,127,2
3,2,21000,60,2021-07-25,8276,158,3
4,0,86057,48,2020-10-04,742,64,4
...,...,...,...,...,...,...,...
49995,0,52499,12,2021-09-28,284,133,49995
49996,2,18043,24,2021-02-15,3275,76,49996
49997,2,5000,36,2021-05-19,7270,3,49997
49998,1,2899509,12,2021-08-24,8333,7,49998


In [9]:
Payments

,payment_amt,due_dt,charge_no,member_no,payment_no
0,2265.716667,2020-02-15,7,3615,324
1,2265.716667,2020-03-15,7,3615,325
2,2265.716667,2020-04-15,7,3615,326
3,2265.716667,2020-05-15,7,3615,327
4,2265.716667,2020-06-15,7,3615,328
...,...,...,...,...,...
58667,104.166667,2020-08-07,49989,7300,1801455
58668,10416.666667,2020-05-04,49991,9091,1801560
58669,10416.666667,2020-06-04,49991,9091,1801561
58670,10416.666667,2020-07-04,49991,9091,1801562


In [10]:
member_no = 907

charges = Charges[Charges.member_no == member_no]
charges

,Charge type,Charge amount,Duration,dt,member_no,provider_no,charge_no
8016,0,95577,48,2021-02-10,907,19,8016
13040,2,4097,48,2020-02-28,907,31,13040
34061,1,1024379,24,2021-11-08,907,31,34061
35409,0,90079,60,2021-05-26,907,106,35409
35565,1,687448,48,2021-03-14,907,85,35565
38877,1,500000,48,2021-09-05,907,135,38877
44159,2,5000,60,2021-01-17,907,147,44159
45079,1,371670,36,2021-09-06,907,40,45079


In [11]:
providers_no = charges.provider_no.values
providers_no

array([ 19,  31,  31, 106,  85, 135, 147,  40])

In [12]:
providers = []
for provider_no in providers_no:
    provider = Providers[Providers.provider_no == provider_no].loc[provider_no]
    providers.append(provider)
providers

[provider_no                          19
 provider_name    PNC Financial Services
 phone_number             (234) 722-1946
 country                   United States
 region_no                            OK
 city                      Oklahoma City
 mail_code                         73121
 street            1908 Peachtree Avenue
 Name: 19, dtype: object,
 provider_no                         31
 provider_name                Citigroup
 phone_number            (360) 440-7176
 country                  United States
 region_no                           MA
 city                           Webster
 mail_code                         1570
 street           194 North Main Street
 Name: 31, dtype: object,
 provider_no                         31
 provider_name                Citigroup
 phone_number            (360) 440-7176
 country                  United States
 region_no                           MA
 city                           Webster
 mail_code                         1570
 street           19

In [13]:
charges_no = charges.charge_no.values
charges_no

array([ 8016, 13040, 34061, 35409, 35565, 38877, 44159, 45079])

In [14]:
payments = []
for charge_no in charges_no:
    payments_no = Payments[Payments.charge_no == charge_no]['payment_no'].values
    payments.append(list(payments_no))
payments

[[], [471420, 471421, 471422, 471423, 471424, 471425], [], [], [], [], [], []]

In [15]:
charges_json = []
for i in range(len(charges_no)):
    charge = charges.iloc[i-1]
    provider = providers[i]
    payments_list = payments[i]
    charge_json = {
        "charge_no": charge.charge_no,
        "category_no": charge['Charge type'],
        #"category_desc": "Travel",
        "charge_dt": charge['dt'],
        "charge_amount": charge['Charge amount'],

        "provider_no": provider.provider_no,
        "provider_name": provider.provider_name,
        "provider_phone_number": provider.phone_number,
        "payments": payments_list
    }
    charges_json.append(charge_json)
    
charges_json

[{'charge_no': 45079,
  'category_no': 1,
  'charge_dt': '2021-09-06',
  'charge_amount': 371670,
  'provider_no': 19,
  'provider_name': 'PNC Financial Services',
  'provider_phone_number': '(234) 722-1946',
  'payments': []},
 {'charge_no': 8016,
  'category_no': 0,
  'charge_dt': '2021-02-10',
  'charge_amount': 95577,
  'provider_no': 31,
  'provider_name': 'Citigroup',
  'provider_phone_number': '(360) 440-7176',
  'payments': [471420, 471421, 471422, 471423, 471424, 471425]},
 {'charge_no': 13040,
  'category_no': 2,
  'charge_dt': '2020-02-28',
  'charge_amount': 4097,
  'provider_no': 31,
  'provider_name': 'Citigroup',
  'provider_phone_number': '(360) 440-7176',
  'payments': []},
 {'charge_no': 34061,
  'category_no': 1,
  'charge_dt': '2021-11-08',
  'charge_amount': 1024379,
  'provider_no': 106,
  'provider_name': 'US Bank',
  'provider_phone_number': '(456) 056-3892',
  'payments': []},
 {'charge_no': 35409,
  'category_no': 0,
  'charge_dt': '2021-05-26',
  'charge_amou

In [16]:
member = Members.loc[lambda x: x.member_no == member_no].loc[member_no]
member

member_no                    907
lastname                  Hansen
firstname               Kristina
phone_number      (066) 692-5965
country            United States
region_no                     CO
city                      Arvada
mail_code                  80003
street          7854 Webster Way
corp_no                       34
prev_balance          314.564225
curr_balance          494.205479
Name: 907, dtype: object

In [17]:
corporation = Corporations.loc[lambda x: x.corp_no == member.corp_no].loc[member.corp_no]
corporation

corp_no                               34
corp_name                Williams-Nguyen
phone_number              (511) 087-8743
country                    United States
region_no                             AR
city                        Fayetteville
mail_code                          72703
street          3425 North Futrall Drive
Name: 34, dtype: object

In [18]:
member_json = {
    "member_no": member_no,
    "lastname": member.lastname,
    "firstname": member.firstname,
    "phone_number": member.phone_number,

    "location":
    {
        "country": member.country,
        "region_no": member.region_no,
        "city": member.city,
        "mail_code": member.mail_code,
        "street": member.street
    },

    "balance":
    {
        "prev_balance": member.prev_balance,
        "curr_balance": member.curr_balance
    },
    "corporation":
    {
        "corp_no": corporation.corp_no,
        "corp_name": corporation.corp_name,
        "region_no": corporation.region_no
    },

    "charges": charges_json
}
member_json

{'member_no': 907,
 'lastname': 'Hansen',
 'firstname': 'Kristina',
 'phone_number': '(066) 692-5965',
 'location': {'country': 'United States',
  'region_no': 'CO',
  'city': 'Arvada',
  'mail_code': 80003,
  'street': '7854 Webster Way'},
 'balance': {'prev_balance': 314.5642248209523,
  'curr_balance': 494.2054786077565},
 'corporation': {'corp_no': 34,
  'corp_name': 'Williams-Nguyen',
  'region_no': 'AR'},
 'charges': [{'charge_no': 45079,
   'category_no': 1,
   'charge_dt': '2021-09-06',
   'charge_amount': 371670,
   'provider_no': 19,
   'provider_name': 'PNC Financial Services',
   'provider_phone_number': '(234) 722-1946',
   'payments': []},
  {'charge_no': 8016,
   'category_no': 0,
   'charge_dt': '2021-02-10',
   'charge_amount': 95577,
   'provider_no': 31,
   'provider_name': 'Citigroup',
   'provider_phone_number': '(360) 440-7176',
   'payments': [471420, 471421, 471422, 471423, 471424, 471425]},
  {'charge_no': 13040,
   'category_no': 2,
   'charge_dt': '2020-02-28

In [19]:
P = []

In [20]:
def get_charges(member_no):
    charges = Charges[Charges.member_no == member_no]
    return charges

def get_member(member_no):
    member = Members.loc[lambda x: x.member_no == member_no].loc[member_no]
    return member

def get_providers(charges):
    providers_no = charges.provider_no.values
    providers = []
    for provider_no in providers_no:
        provider = Providers[Providers.provider_no == provider_no].loc[provider_no]
        providers.append(provider)
    return providers

def get_payments(charges):
    charges_no = charges.charge_no.values
    payments = []
    for charge_no in charges_no:
        payments_no = Payments[Payments.charge_no == charge_no]['payment_no'].values
        payments.append(list(payments_no))
    return payments

def get_charges_json(member_no):
    charges = get_charges(member_no)
    providers = get_providers(charges)
    payments = get_payments(charges)
    
    charges_json = []
    for i in range(len(charges)):
        charge = charges.iloc[i-1]
        provider = providers[i]
        payments_list = payments[i]
        json = {
            "charge_no": charge.charge_no,
            "category_no": charge['Charge type'],
            "category_desc": get_category_name(charge['Charge type']),
            "charge_dt": charge['dt'],
            "charge_amount": charge['Charge amount'],

            "provider_no": provider.provider_no,
            "provider_name": provider.provider_name,
            "provider_phone_number": provider.phone_number,
            "provider_city": provider.city,
            "payments": payments_list
        }
        charges_json.append(json)

    return charges_json

def get_category_name(category_no):
    return Categories.loc[lambda x: x.category_no == category_no].category_name.loc[category_no]

def get_corporation(member):
    corporation = Corporations.loc[lambda x: x.corp_no == member.corp_no].loc[member.corp_no]
    return corporation

def get_member_json(member_no):
    charges_json = get_charges_json(member_no)
    member = get_member(member_no)
    corporation = get_corporation(member)
    
    member_json = {
        "member_no": member_no,
        "lastname": member.lastname,
        "firstname": member.firstname,
        "phone_number": member.phone_number,

        "location":
        {
            "country": member.country,
            "region_no": member.region_no,
            "city": member.city,
            "mail_code": member.mail_code,
            "street": member.street
        },
        
        "balance":
        {
            "prev_balance": member.prev_balance,
            "curr_balance": member.curr_balance
        },

        
        "corporation":
        {
            "corp_no": corporation.corp_no,
            "corp_name": corporation.corp_name,
            "region_no": corporation.region_no
        },

        "charges": charges_json
    }
    return member_json

def get_members_json():
    return Members.member_no.apply(lambda x: get_member_json(x)).to_list()
    # On utilise .uniques() pour éviter les doublons

In [21]:
def get_providers_json():
    providers = [
        {
            "provider_no": provider.provider_no,
            "provider_name": provider.provider_name,
            "location":
            {
                "country": provider.country,
                "region_no": provider.region_no,
                "city": provider.city,
                "mail_code": provider.mail_code
            }
        } for (_,provider) in Providers.iterrows()]
    return providers

def get_payments_json():
    payments = [{
            "payment_no": payment.payment_no,
            "payment_dt": payment.due_dt,
            "payment_amt": payment.payment_amt
        } for (_,payment) in Payments.iterrows()]
    return payments

def final_json():
    return {
        'members': get_members_json(),
        'providers': get_providers_json(),
        'payments': get_payments_json()
    }

# Class pour résoudre les problèmes d'encoding entre numpy et le module json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def _export_mongo(data, filename, collection_name):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile, cls=NpEncoder, indent=4)

    client = MongoClient( mongoDBUrl, port )
    db = client['db_credit']
    collection = db[collection_name]
    #indexName = collection_name.replace('s','')+'_no'
    #print(collection)
    #print(indexName)
    #collection.createIndex({indexName:1})

    with open(filename) as f:
        file_data = json.load(f)
        collection.insert_many(file_data)
        client.close()
        
def members_export_mongo():
    data = get_members_json()
    filename = 'members.json'
    collection_name = 'members'
    _export_mongo(data, filename, collection_name)
    
def providers_export_mongo():
    data = get_providers_json()
    filename = 'providers.json'
    collection_name = 'providers'
    _export_mongo(data, filename, collection_name)
    
def payments_export_mongo():
    data = get_payments_json()
    filename = 'payments.json'
    collection_name = 'payments'
    _export_mongo(data, filename, collection_name)
    
def export_mongo():
    members_export_mongo()
    providers_export_mongo()
    payments_export_mongo()

**À cette étape on va créer les index directement en ligne de commande**

In [22]:
export_mongo()

In [23]:
client = MongoClient( mongoDBUrl, port )
db = client['db_credit']
db.list_collection_names()

['providers', 'members', 'payments']

In [24]:
collection = db['members']
collection.count_documents({})

10000

In [25]:
collection = db['payments']
collection.count_documents({})

58672

In [26]:
collection = db['providers']
collection.count_documents({})

200

In [27]:
client.close()